In [ ]:
from selenium import webdriver
import time
import requests
import os
import matplotlib.pyplot as plt
import cv2
from bs4 import BeautifulSoup
import datetime
import pandas as pd

In [ ]:
#開啟瀏覽器進入FB
def OpenBrowser():
    print('OpenBrowser')
    profile=webdriver.FirefoxProfile()
    
    #避免跳出視窗
    profile.set_preference("dom.webnotifications.enabled",False)
    profile.update_preferences()
    driver=webdriver.Firefox(firefox_profile=profile)
    
    #切換至臉書登入頁面
    driver.get("https://www.facebook.com/login")
    time.sleep(3)
    print('OpenBrowser......finish\n')
    return driver


In [ ]:
#登入爬蟲帳號
def LoginFB(USERNAME,PASSWORD,driver):
    print('LoginFB')
    
    #填寫帳密&按登入鍵
    driver.find_element_by_id("email").send_keys(USERNAME)
    driver.find_element_by_id("pass").send_keys(PASSWORD)
    driver.find_element_by_id("loginbutton").click()
    time.sleep(3)
    print('LoginFB......finish\n')

In [ ]:
#切換至目標頁
def Go_to_Target_Page(target,driver):
    print('Go_to_Target_Page')
    driver.get("https://www.facebook.com/"+target)
    time.sleep(3)
    print('Go_to_Target_Page......finish\n')

In [ ]:
#取得html資訊
def Get_HTML(driver,rolling):
    print('Get_HTML')
    
    #向下捲動更新頁面
    for i in range(rolling):
        driver.execute_script("window.scrollTo(0, {})".format(4000 * (i + 1)))
        time.sleep(2)
    
    #點開貼文的'更多'
    see_more_list=driver.find_elements_by_class_name("see_more_link")
    more=0
    for item in see_more_list:
        try:
            item.click()
            more+=1
            time.sleep(2)
        except:
            pass
    print("有 ",len(see_more_list)," 更多")
    print("成功點開 ",more," 個")
    
    #取得body
    htmltext=driver.page_source
    soup=BeautifulSoup(htmltext,'html.parser')
    body=soup.find('body')
    print('Get_HTML......finish\n')
    return body

In [ ]:
#取得FB ID
def Get_ID(body):
    print('Get_ID')
    FB_ID=body.find('a',{'class':'coverWrap coverImage'})
    DataReferrerid=FB_ID.get('data-referrerid')
    print('FB ID: ',DataReferrerid)
    print('Get_ID......finish\n')
    return DataReferrerid

In [ ]:
#取得貼文內容
def Get_Post(body,target):
    print('Get_Post')
    post=body.select('div[data-testid="post_message"]')
    post_text=[]
    
    #去除 '更多','......'等文字
    for i in range(len(post)):
        try:
            [item.extract() for item in post[i].find('span',{'class':"see_more_link_inner"})]
            [item.extract() for item in post[i].find('span',{'class':"text_exposed_hide"})]
        except:
            pass
        post_text.append(post[i].text)
    
    #存成CSV
    post_list=pd.DataFrame()
    post_list['text']=post_text
    if not os.path.isdir(str(target)):
        os.mkdir(str(target))
    post_list.to_csv(str(target)+'/post_list.csv',index=False)
    print('Get_Post......finish\n')

In [ ]:
#取得封面頁
def Get_CoverBorder(body,target):
    print('Get_CoverBorder')
    coverBorder=body.find('img',{'class':'coverPhotoImg photo img'})
    cover_URL=coverBorder.get('src')
    #print(cover_URL)

    r=requests.get(cover_URL)
    if not os.path.isdir(target):
        os.mkdir(target)
    with open(str(target)+'/coverBorder.jpg','wb') as f:
        f.write(r.content)
    print('Get_CoverBorder......finish\n')

In [ ]:
#取得大頭貼
def Get_BigHead(body,target):
    print('Get_BigHead')
    BigHead=body.find('img',{'class':'_11kf img'})
    BigHead_URL=BigHead.get('src')
    #print(BigHead_URL)

    r=requests.get(BigHead_URL)
    if not os.path.isdir(target):
        os.mkdir(target)
    with open(str(target)+'/BigHead.jpg','wb') as f:
        f.write(r.content)
    print('Get_BigHead......finish\n')

In [ ]:
#切換至照片頁
def Go_to_Photo_Page(driver,target):
    print('Go_to_Photo_Page')
    driver.get("https://www.facebook.com/"+target+"/photos_all")
    time.sleep(3)
    print('Go_to_Photo_Page......finish\n')

In [ ]:
#取得照片
def Get_Photos(driver,target,rolling):
    print('Get_Photos')
    
    #向下捲動更新頁面
    for i in range(rolling):
        driver.execute_script("window.scrollTo(0, {})".format(4000 * (i + 1)))
        time.sleep(3)
        
    photo_htmltext=driver.page_source
    photo_soup=BeautifulSoup(photo_htmltext,'html.parser')
    photo_body=photo_soup.find('body')
    photos=photo_body.find_all('li',{'class':'fbPhotoStarGridElement fbPhotoStarGridNonStarred focus_target _53s fbPhotoCurationControlWrapper'})
    print("共有 ",len(photos)," 張圖片")
    
    path=str(target)+'/photo'
    if not os.path.isdir(path):
        os.mkdir(path)
    
    #根據圖片URL下載每張圖片
    fail_count=0
    for i in range(len(photos)):
        try:
            with open(path+'/photo_'+str(i)+'.jpg','wb') as f:
                r=requests.get(photos[i].get('data-starred-src'))
                f.write(r.content)
        except:
            fail_count+=1
    print(len(photos)-fail_count," 張圖片下載成功")
    print(fail_count," 張圖片下載失敗")
    print('Get_Photos......finish\n')

In [ ]:
#切換至好友頁
def Go_to_Friend_Page(driver,target):
    print('Go_to_Friend_Page')
    driver.get("https://www.facebook.com/"+target+"/friends_all")
    time.sleep(3)
    print('Go_to_Friend_Page......finish\n')

In [ ]:
#取得好友清單 (名字,ID)
def Get_Friends(driver,target,rolling):
    print('Get_Friends')
    
    #向下捲動更新頁面
    for i in range(rolling):
        driver.execute_script("window.scrollTo(0, {})".format(4000 * (i + 1)))
        time.sleep(2)
    
    friend_htmltext=driver.page_source
    friend_soup=BeautifulSoup(friend_htmltext,'html.parser')
    friend_body=friend_soup.find('body')
    
    friends=friend_body.find_all('li',{'class':'_698'})
    print("共有 ",len(friends)," 位好友")
    friend_name=[]
    friend_ID=[]
    fail=0
    
    #分別處理 '共同好友' '可加好友' '不開放加好友'
    for i in range(len(friends)):
        friend_name.append(friends[i].find('div',{'class':'fsl fwb fcb'}).text)
        try:
            friend_ID.append(friends[i].find('button',{'data-floc':'friends_tab'}).get('data-profileid'))

        except:
            try:
                friend_ID.append(friends[i].find('a',{'data-floc':'friends_tab'}).get('data-profileid'))
            except:
                friend_ID.append("None")
                fail+=1
    print(fail," 取得失敗")
    
    #存成CSV
    friend_list=pd.DataFrame()
    friend_list['name']=friend_name
    friend_list['ID']=friend_ID
    if not os.path.isdir(target):
        os.mkdir(target)
    friend_list.to_csv(str(target)+'/friend_list.csv',index=False)
    print('Get_Friends......finish\n')

In [ ]:
#全部執行一遍

driver=OpenBrowser()               #開啟瀏覽器進入FB

rolling=15                         #向下捲動的次數

USERNAME="USERNAME"                #爬蟲帳密
PASSWORD="PASSWORD"

target='target'                    #username like "xiao.ming.wang or ID"

LoginFB(USERNAME,PASSWORD,driver)  #登入爬蟲帳號


Go_to_Target_Page(target,driver)   #切換至目標頁

body=Get_HTML(driver,rolling)      #取得html資訊

ID=Get_ID(body)                    #取得FB ID

Get_Post(body,target)              #取得貼文內容

Get_CoverBorder(body,target)       #取得封面頁

Get_BigHead(body,target)           #取得大頭貼

Go_to_Photo_Page(driver,target)    #切換至照片頁

Get_Photos(driver,target,rolling)  #取得照片

Go_to_Friend_Page(driver,target)   #切換至好友頁

Get_Friends(driver,target,rolling) #取得好友清單 (名字,ID)

print("-----Crawler Mission Completed-----")